Прошу дать еще немного времени

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('/Users/evgeniya/Desktop/Рекомендательные системы/2/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
my_result = data_test.groupby('user_id')['item_id'].unique().reset_index()
my_result.columns=['user_id', 'actual']
my_result['actual'] = my_result['actual'].apply(lambda x: list(x))
my_result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights.iloc[:,0])
    probability = np.array(items_weights.iloc[:,1])
    recs = np.random.choice(items, size=n, replace=True, p=probability)
    
    return recs.tolist()

Сделайте предсказания

In [6]:
%%time

sales_log = data_train.groupby('item_id')['sales_value'].sum().reset_index()    

sales_log['sales_value'] = np.log(sales_log['sales_value'], out=np.zeros_like(sales_log['sales_value']), where=(sales_log['sales_value'] !=0))

sales_log.loc[sales_log['sales_value'] < 0, 'sales_value'] = 0

sales_log['sales_value'] = sales_log['sales_value']/(sales_log['sales_value'].sum())

my_result['weighted_random_recs_1'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales_log, n=5))

CPU times: user 1.78 s, sys: 169 ms, total: 1.95 s
Wall time: 1.79 s


In [7]:
%%time

sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
sales['sales_value'] = sales['sales_value']/(sales['sales_value'].sum())

my_result['weighted_random_recs_2'] = my_result['user_id'].apply(lambda x: weighted_random_recommendation(sales, n=5))

CPU times: user 1.73 s, sys: 5.34 ms, total: 1.74 s
Wall time: 1.74 s


In [8]:
my_result.head(5)

,user_id,actual,weighted_random_recs_1,weighted_random_recs_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9829577, 1046825, 905325, 888625, 1039495]","[1013578, 913894, 1091809, 6534178, 1013677]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13382461, 9296877, 1047182, 844740, 867634]","[908224, 1134214, 971922, 893118, 10204556]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1126704, 1015145, 10284944, 9858848, 1726339]","[9553363, 6534178, 847790, 8090536, 883404]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[967994, 8090612, 6602711, 10344969, 9859133]","[839346, 13190294, 1081177, 1013167, 886621]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1009966, 6919544, 941024, 5569261, 826784]","[9799184, 10204605, 820361, 1045923, 1103105]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [9]:
result = pd.read_csv('/Users/evgeniya/Desktop/Рекомендательные системы/2/preds.csv') # закгрузка predict с семианара

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13039087, 1190490, 6552030, 9884683, 1617215]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965430, 664229, 829690, 4519071, 1015781]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [10]:
type(result['actual'][0])

str

In [11]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result[a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [12]:
result['weighted_random_recs_1'] = my_result['weighted_random_recs_1']
result['weighted_random_recs_2'] = my_result['weighted_random_recs_2']

result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recs_1,weighted_random_recs_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13039087, 1190490, 6552030, 9884683, 1617215]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[9829577, 1046825, 905325, 888625, 1039495]","[1013578, 913894, 1091809, 6534178, 1013677]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[965430, 664229, 829690, 4519071, 1015781]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[13382461, 9296877, 1047182, 844740, 867634]","[908224, 1134214, 971922, 893118, 10204556]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2664434, 74980, 864943, 9337133, 77162]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[1126704, 1015145, 10284944, 9858848, 1726339]","[9553363, 6534178, 847790, 8090536, 883404]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[907426, 10149374, 1050777, 16058940, 6381881]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[967994, 8090612, 6602711, 10344969, 9859133]","[839346, 13190294, 1081177, 1013167, 886621]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1112073, 9529550, 1172420, 84012, 7432861]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[1009966, 6919544, 941024, 5569261, 826784]","[9799184, 10204605, 820361, 1045923, 1103105]"


In [13]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [18]:
# Precision@5 для random_recommendation

result.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1).mean()

0.0008814887365328109

In [19]:
# Precision@5 для popular_recommendation

result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()

0.15523996082272082

In [20]:
# Precision@5 для itemitem

result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()

0.13692458374142857

In [21]:
# Precision@5 для cosine

result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()

0.13290891283055686

In [22]:
# Precision@5 для tfidf

result.apply(lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1).mean()

0.1389813907933383

In [23]:
# Precision@5 для own_purchases

result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()

0.17969311132876015

In [24]:
# Precision@5 для weighted_random_recs_1

result.apply(lambda x: precision_at_k(x['weighted_random_recs_1'], x['actual'],  5), axis=1).mean()

0.0017629774730656225

In [25]:
# Precision@5 для weighted_random_recs_2

result.apply(lambda x: precision_at_k(x['weighted_random_recs_2'], x['actual'],  5), axis=1).mean()

0.02007835455435849

own_purchases показал наилучшее качество

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [15]:
# your_code

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.